# Combining raw dataframes information together to form a master

In [ ]:
"""
using waiting_times as the base for master as it contains 
all relevant information for attraction specific wait times
"""
import pandas as pd

master = pd.read_parquet("../data/raw_data/waiting_times.parquet")


# ===================================
# |  link attraction park merging   |
# ===================================

# link attraction park to label which attraction belongoning to which park
link_attraction_park = pd.read_parquet("../data/processed_data/link_attraction_park.parquet")
master_merged_1 = master.merge(link_attraction_park, left_on = 'ENTITY_DESCRIPTION_SHORT', right_on = 'ATTRACTION', how = 'left')

# replacing ENTITY_DESCRIPTION_SHORT with ATTRACTION for clarity
master_merged_1.drop(columns = ['ENTITY_DESCRIPTION_SHORT'], inplace = True)  


# ===================================
# |           attendance            |
# ===================================

# link attendance to mark attendance for each park and date. this way wait times will have daily attendance
attendance = pd.read_parquet("../data/raw_data/attendance.parquet")

# converting datetimes to datetime format
attendance['USAGE_DATE'] = pd.to_datetime(attendance['USAGE_DATE'])
master_merged_1['WORK_DATE'] = pd.to_datetime(master_merged_1['WORK_DATE'])

master_merged_2 = master_merged_1.merge(attendance, left_on = ['PARK', 'WORK_DATE'], right_on = ['FACILITY_NAME', 'USAGE_DATE'], how = 'left')
#! there are a bunch of dates that dont have attendance data, keep for now

# removing FACILITY_NAME and USAGE_DATE since redundant after merge
master_merged_2.drop(columns = ['FACILITY_NAME', 'USAGE_DATE'], inplace = True)
master_merged_2


# ===================================
# |         weather data            |
# ===================================

# link weather data to mark weather for each park and date. this way wait times will have daily weather
weather_data = pd.read_parquet("../data/raw_data/weather_data.parquet")

# weather data time is in iso format and UTC, converting it to datetime with timezone offset
weather_data['dt_iso'] = pd.to_datetime(weather_data['dt'], unit = 's') + pd.to_timedelta(weather_data['timezone'], unit='s')

master_merged_3 = master_merged_2.merge(weather_data, left_on = 'WORK_DATE', right_on = 'dt_iso', how = 'left')

# removing all redundant datetime/ timeszone/ location columns
master_merged_3.drop(columns = ['dt', 'dt_iso', 'timezone', 'city_name', 'lat', 'lon'], inplace = True)


# ===================================
# |        entity schedule          |
# ===================================

# link entity schedule to indicate when there are closures for whichever attraction
entity_schedule = pd.read_parquet("../data/raw_data/entity_schedule.parquet")

# creating a subset so i dont merge the entire entity schedule with redundant columns
entity_schedule_subset = entity_schedule[['REF_CLOSING_DESCRIPTION', 'ENTITY_DESCRIPTION_SHORT', 'WORK_DATE']].copy()

# converting datetimes to datetime format
entity_schedule_subset['WORK_DATE'] = pd.to_datetime(entity_schedule_subset['WORK_DATE'])

master_merged_4 = master_merged_3.merge(entity_schedule_subset, left_on = ['ATTRACTION', 'WORK_DATE'], right_on = ['ENTITY_DESCRIPTION_SHORT', 'WORK_DATE'], how = 'left')

# dropping redundant merge columns
master_merged_4.drop(columns = [ 'ENTITY_DESCRIPTION_SHORT'], inplace = True)

In [ ]:
#master_merged_4.to_parquet('../data/processed_data/master.parquet', index=False)

# Adding Months feature from WORK_DATE

In [45]:
#turning WORK_DATE into datetime then extracting months as a feature
master_merged_4['WORK_DATE'] = pd.to_datetime(master_merged_4['WORK_DATE'])
master_merged_4['month'] = master_merged_4['WORK_DATE'].dt.month_name()

In [46]:
"""
filtering out extreme outliers for numerical features by
removing any data points outside of 3*IQR for each numerical column
"""
df_clean= df.copy()

#get the numerical columns
numerical_cols = df_clean.select_dtypes(include=['number']).columns

for col in numerical_cols:
    Q1  = df_clean[col].quantile(0.25)
    Q3  = df_clean[col].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 3*IQR
    upper_bound = Q3 + 3*IQR

    #count outliers before removal
    outlier_count = len(df_clean[(df_clean[col] < lower_bound) | (df_clean[col] > upper_bound)])
    print(f"Column: {col}, has {outlier_count} outliers")

    #remove the outliers
    df_clean = df_clean[(df_clean[col] >= lower_bound) & (df_clean[col] <= upper_bound)]
display(df_clean.head())

,WORK_DATE,DEB_TIME_x,DEB_TIME_HOUR,FIN_TIME_x,WAIT_TIME_MAX,NB_UNITS,GUEST_CARRIED,CAPACITY,ADJUST_CAPACITY,OPEN_TIME,...,weather_id,weather_main,weather_description,weather_icon,REF_CLOSING_DESCRIPTION,ENTITY_DESCRIPTION_SHORT,ENTITY_TYPE,DEB_TIME_y,FIN_TIME_y,UPDATE_TIME
0,2018-01-01,2018-01-01 21:00:00.000,21,2018-01-01 21:15:00.000,0,2.0,0.0,0.000,0.00,0,...,802,Clouds,scattered clouds,03n,NaN,Roller Coaster,ATTR,2018-01-01 10:00:00.000,2018-01-01 17:44:00.000,2018-01-02 07:46:01.000
1,2018-01-01,2018-01-01 19:30:00.000,19,2018-01-01 19:45:00.000,5,18.0,148.0,254.749,254.75,15,...,802,Clouds,scattered clouds,03n,NaN,Bumper Cars,ATTR,2018-01-01 08:30:00.000,2018-01-01 21:05:00.000,2018-01-02 07:51:31.000
2,2018-01-01,2018-01-01 22:30:00.000,22,2018-01-01 22:45:00.000,0,1.0,0.0,0.000,0.00,0,...,802,Clouds,scattered clouds,03n,NaN,Rapids Ride,ATTR,2018-01-01 09:29:00.000,2018-01-01 17:08:00.000,2018-01-02 07:51:31.000
3,2018-01-01,2018-01-01 12:45:00.000,12,2018-01-01 13:00:00.000,5,1.0,46.0,250.001,250.00,15,...,802,Clouds,scattered clouds,03n,NaN,Crazy Dance,ATTR,2018-01-01 09:12:00.000,2018-01-01 22:00:00.000,2018-01-02 07:43:47.000
4,2018-01-01,2018-01-01 17:00:00.000,17,2018-01-01 17:15:00.000,5,15.0,92.0,211.500,198.25,15,...,802,Clouds,scattered clouds,03n,NaN,Skyway,ATTR,2018-01-01 10:00:00.000,2018-01-01 19:00:00.000,2018-01-02 07:55:16.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3509319,2022-08-18,2022-08-18 18:45:00.000,18,2022-08-18 19:00:00.000,0,0.0,0.0,0.000,0.00,0,...,804,Clouds,overcast clouds,04n,Fermeture Réhab,Himalaya Ride,ATTR,2022-08-18 23:59:00.000,2022-08-18 23:59:00.000,2022-08-19 07:56:33.000
3509320,2022-08-18,2022-08-18 10:15:00.000,10,2022-08-18 10:30:00.000,0,0.0,0.0,0.000,0.00,0,...,804,Clouds,overcast clouds,04n,Fermeture Réhab,Crazy Dance,ATTR,2022-08-18 23:59:00.000,2022-08-18 23:59:00.000,2022-08-19 07:56:29.000
3509321,2022-08-18,2022-08-18 09:15:00.000,9,2022-08-18 09:30:00.000,0,0.0,0.0,0.000,0.00,0,...,804,Clouds,overcast clouds,04n,Fermeture Réhab,Crazy Dance,ATTR,2022-08-18 23:59:00.000,2022-08-18 23:59:00.000,2022-08-19 07:56:29.000
3509322,2022-08-18,2022-08-18 20:30:00.000,20,2022-08-18 20:45:00.000,0,0.0,0.0,0.000,0.00,0,...,804,Clouds,overcast clouds,04n,Fermeture Réhab,Giga Coaster,ATTR,2022-08-18 23:59:00.000,2022-08-18 23:59:00.000,2022-08-19 07:56:35.000


standard normal transformation

$$ (x-µ

In [50]:
#finding imbalance categories

#converting dateetime strings into datetime dtype
df_clean['DEB_TIME']= pd.to_datetime(df_clean['DEB_TIME'])
df_clean['FIN_TIME']= pd.to_datetime(df_clean['FIN_TIME'])

categorical_cols = df_clean.select_dtypes(include=['object', 'category']).columns

imbalance_cols = []

for col in categorical_cols:
    value_counts = df_clean[col].value_counts()
    percentages = value_counts / len(df_clean) * 100

    #check highest occuring category and lowest occurring category to find imbalance ratio
    imbalance_ration =  (percentages.max()-percentages.min())

    #check if imbalance ratio is 5 > times
    if imbalance_ration > 5:
        imbalance_cols.append(col)
print(f"Imbalance columns: {imbalance_cols}")
    

Imbalance columns: ['PARK', 'weather_main', 'weather_description', 'weather_icon', 'REF_CLOSING_DESCRIPTION', 'month']


/var/folders/bn/nfd56p353h9gnytlmplrwn880000gn/T/ipykernel_36671/1102225279.py:7: Pandas4Warning: For backward compatibility, 'str' dtypes are included by select_dtypes when 'object' dtype is specified. This behavior is deprecated and will be removed in a future version. Explicitly pass 'str' to `include` to select them, or to `exclude` to remove them and silence this warning.
See https://pandas.pydata.org/docs/user_guide/migration-3-strings.html#string-migration-select-dtypes for details on how to write code that works with pandas 2 and 3.
  categorical_cols = df_clean.select_dtypes(include=['object', 'category']).columns


In [51]:
train_df.columns

Index(['DEB_TIME', 'DEB_TIME_HOUR', 'FIN_TIME', 'WAIT_TIME_MAX', 'NB_UNITS',
       'GUEST_CARRIED', 'CAPACITY', 'ADJUST_CAPACITY', 'OPEN_TIME', 'UP_TIME',
       'DOWNTIME', 'NB_MAX_UNIT', 'ATTRACTION', 'PARK', 'attendance', 'temp',
       'dew_point', 'feels_like', 'temp_min', 'temp_max', 'pressure',
       'humidity', 'wind_speed', 'wind_deg', 'clouds_all', 'weather_id',
       'weather_main', 'weather_description', 'weather_icon',
       'REF_CLOSING_DESCRIPTION'],
      dtype='str')

In [52]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# split our data before scaling to avoid data leakage
train_df, test_df = train_test_split(df_clean, 
                                     test_size=0.4, 
                                     random_state=42,
                                     stratify=df_clean[['PARK','weather_main', 'weather_description','weather_icon','REF_CLOSING_DESCRIPTION']])

val_df, test_df = train_test_split(test_df, 
                                     test_size=0.5, 
                                     random_state=42,
                                     stratify=test_df[['PARK','weather_main', 'weather_description','weather_icon','REF_CLOSING_DESCRIPTION']]) 

print(f"Original Shape: {df_clean.shape}")
print(f"Train Set Shape: {train_df.shape}")
print(f"Validation Set hape: {val_df.shape}")
print(f"Test Set Shape: {test_df.shape}")

numerical_cols = df_clean.select_dtypes(include=['number']).columns

Original Shape: (1893178, 32)
Train Set Shape: (1135906, 32)
Validation Set hape: (378636, 32)
Test Set Shape: (378636, 32)


In [55]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

numerical_cols = train_df.select_dtypes(include=['number']).columns
# even though its numeric it should be treated as categorical
numerical_cols = numerical_cols.drop('DEB_TIME_HOUR')

# scale fit on train set ONLY
train_df_scaled = scaler.fit_transform(train_df[numerical_cols])

# scale transform on val and test set using the same scaler fit on train set
val_df_scaled = scaler.transform(val_df[numerical_cols])
test_df_scaled = scaler.transform(test_df[numerical_cols])

train_df_scaled = pd.DataFrame(train_df_scaled, columns = numerical_cols, index = train_df.index)
val_df_scaled = pd.DataFrame(val_df_scaled, columns = numerical_cols, index = val_df.index)
test_df_scaled = pd.DataFrame(test_df_scaled, columns = numerical_cols, index = test_df.index)

print(f"Train set shape: {train_df_scaled.shape}")
print(f"Validation set shape: {val_df_scaled.shape}")
print(f"Test set shape: {test_df_scaled.shape}")

Train set shape: (1135906, 21)
Validation set shape: (378636, 21)
Test set shape: (378636, 21)


# Categorical Enconding
One hot encoding because we dont have any ordinal categories for ordinal or binary encoding

In [56]:
from sklearn.preprocessing import OneHotEncoder

categorical_cols = df_clean.select_dtypes(include=['object', 'category']).columns.tolist()
# adding hours to categorical
categorical_cols.append('DEB_TIME_HOUR')

# dropping redundant merge columns
master_merged_4.drop(columns = ['WORK_DATE', 'ENTITY_DESCRIPTION_SHORT'], inplace = True)

/var/folders/bn/nfd56p353h9gnytlmplrwn880000gn/T/ipykernel_36671/3024805385.py:3: Pandas4Warning: For backward compatibility, 'str' dtypes are included by select_dtypes when 'object' dtype is specified. This behavior is deprecated and will be removed in a future version. Explicitly pass 'str' to `include` to select them, or to `exclude` to remove them and silence this warning.
See https://pandas.pydata.org/docs/user_guide/migration-3-strings.html#string-migration-select-dtypes for details on how to write code that works with pandas 2 and 3.
  categorical_cols = df_clean.select_dtypes(include=['object', 'category']).columns.tolist()


Train set shape: (1135906, 70)
Validation set shape: (378636, 70)
Test set shape: (378636, 70)
